## SALib Sensitivity analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyNetLogo

# Import variance-based sensitivity analysis
from SALib.sample import saltelli
from SALib.analyze import sobol

In [2]:
problem = {
    'num_vars':6,
    'names': ['random-seed',
            'grass-regrowth-time',
            'sheep-gain-from-food',
            'wolf-gain-from-food',
            'sheep-reproduce',
            'wolf-reproduce'], 
  'bounds': [[1, 100000],
             [20., 40.], 
             [2., 8.], 
             [16., 32.],
             [2., 8.],
             [2., 8.]]
}

In [3]:
netlogo = pyNetLogo.NetLogoLink(gui=False)
netlogo.load_model(r'Wolf Sheep Predation_v6.nlogo')

In [4]:
n = 1000
param_values = saltelli.sample(problem, n, calc_second_order=True)

In [5]:
param_values.shape

(14000, 6)

In [ ]:
results = pd.DataFrame(columns=['Avg. sheep', 'Avg. wolves'])

In [ ]:
import time

t0 = time.time()
for run in range(param_values.shape[0]):
    #Set the input parameters
    for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(param_values[run,i]))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, param_values[run,i]))
    netlogo.command('setup')
    counts = netlogo.repeat_report(['count sheep', 'count wolves'], 100)
    
    results.loc[run, 'Avg. sheep'] = counts['count sheep'].values.mean()
    results.loc[run, 'Avg. wolves'] = counts['count wolves'].values.mean()
    
    if run%10 == 0:
        elapsed1 = time.time()-t0
        print('Iteration: ',run ,'times; ','Elapsed: ',elapsed1,' seconds')
        
elapsed = time.time() - t0

Iteration:  0 times;  Elapsed:  1.0658960342407227  seconds
Iteration:  10 times;  Elapsed:  3.8353471755981445  seconds
Iteration:  20 times;  Elapsed:  5.4866485595703125  seconds
Iteration:  30 times;  Elapsed:  7.152511358261108  seconds
Iteration:  40 times;  Elapsed:  9.002635478973389  seconds
Iteration:  50 times;  Elapsed:  10.502870321273804  seconds
Iteration:  60 times;  Elapsed:  11.937369346618652  seconds
Iteration:  70 times;  Elapsed:  13.67452597618103  seconds
Iteration:  80 times;  Elapsed:  15.230409145355225  seconds
Iteration:  90 times;  Elapsed:  16.847043991088867  seconds
Iteration:  100 times;  Elapsed:  18.340049505233765  seconds
Iteration:  110 times;  Elapsed:  19.73359704017639  seconds
Iteration:  120 times;  Elapsed:  21.147326707839966  seconds
Iteration:  130 times;  Elapsed:  22.46329665184021  seconds
Iteration:  140 times;  Elapsed:  24.064109802246094  seconds
Iteration:  150 times;  Elapsed:  25.528659105300903  seconds
Iteration:  160 times;  

In [ ]:
print(elapsed)

In [ ]:
netlogo2 = pyNetLogo.NetLogoLink(gui=True)
netlogo2.load_model(r'Wolf Sheep Predation_v6.nlogo')